In [112]:
'''Libraries'''
import numpy as np, torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader, Subset
from laplace import Laplace
import matplotlib.pyplot as plt
import laplace
import pickle
import os
import time


In [7]:
'''Dataset Loader for Digits dataset'''
np.random.seed(0)
torch.manual_seed(0)

# Data
digits = load_digits()
X, y = digits.data.astype(np.float32), digits.target.astype(np.int64)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

scaler = StandardScaler().fit(Xtr)
Xtr, Xte = scaler.transform(Xtr), scaler.transform(Xte)

Xtr_t = torch.from_numpy(Xtr)
ytr_t = torch.from_numpy(ytr)
Xte_t = torch.from_numpy(Xte)
yte_t = torch.from_numpy(yte)

train_set = TensorDataset(Xtr_t, ytr_t)
print(train_set)
n_train = len(train_set)

Task 1

In [ ]:
def count_parameters(model, only_trainable=False):
    """
    모델의 파라미터 개수를 반환합니다.
    only_trainable=True이면 requires_grad=True인 파라미터만 셉니다.
    """
    params = (p for p in model.parameters() if (not only_trainable) or p.requires_grad)
    return sum(p.numel() for p in params)

def get_models():
    models = []

    # 1. 가장 단순한 모델
    models.append(nn.Linear(64, 10))

    # 2. 파라미터 2배: Linear + Linear
    models.append(nn.Sequential(
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    ))

    # 3. 더 깊게: Linear + Linear + Linear
    models.append(nn.Sequential(
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    ))

    # 4. 더 깊고 넓게
    models.append(nn.Sequential(
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    # 5. BatchNorm 추가
    models.append(nn.Sequential(
        nn.Linear(64, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    ))

    # 6. Dropout 추가
    models.append(nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    # 7. 더 깊게, 더 넓게
    models.append(nn.Sequential(
        nn.Linear(64, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    # 8. 더 많은 레이어와 BatchNorm, Dropout
    models.append(nn.Sequential(
        nn.Linear(64, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    # 9. 더 깊고 넓게, 활성화 다양화
    models.append(nn.Sequential(
        nn.Linear(64, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.Tanh(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    # 10. 가장 큰 모델
    models.append(nn.Sequential(
        nn.Linear(64, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    ))

    return models

def evaluate(loader, model):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in loader:
                outputs = model(inputs)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        return correct / total

def cumulative_explained_ratio(val, alpha=0.9):
    """
    val: list or np.array of eigenvalues (can be positive/negative)
    alpha: target cumulative proportion (e.g., 0.9)
    
    Returns:
        ratio (float): index/p where cumulative sum first exceeds alpha
        idx (int): the actual index achieving it
    """
    val = np.array(val, dtype=float)
    # 음수 값은 curvature 설명에 기여하지 않도록 제거 (필요시 옵션화 가능)
    val = np.maximum(val, 0)
    
    if np.sum(val) == 0:
        return 0.0, 0

    sorted_vals = np.sort(val)[::-1]  # 내림차순 정렬
    cumvals = np.cumsum(sorted_vals)
    total = cumvals[-1]
    threshold = alpha * total

    idx = np.searchsorted(cumvals, threshold)
    ratio = (idx + 1) / len(val)
    return ratio, idx + 1

In [ ]:
n_repeats = 10
alpha = 0.9
results = {}

for repeat in range(n_repeats):
    print(f"\n=== Repetition {repeat+1}/{n_repeats} ===")
    
    # 💡 매 반복마다 fresh initialization
    models = get_models()

    for model in models:
        n_params = count_parameters(model)
        x_val = np.log(n_params)

        print(f"\nTraining model with {n_params} parameters (repeat {repeat+1})")

        torch.manual_seed(repeat)
        np.random.seed(repeat)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        test_loader = DataLoader(TensorDataset(Xte_t, yte_t), batch_size=256, shuffle=False)
        train_eval_loader = DataLoader(train_set, batch_size=256, shuffle=False)

        n_epochs_monitor = 30
        for epoch in range(n_epochs_monitor):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

        # after training:
        la = Laplace(model, 'classification', subset_of_weights='all', hessian_structure='diag')
        la.fit(train_loader)
        val = la.H.cpu().numpy()

        ratio, idx = cumulative_explained_ratio(val, alpha=alpha)

        # 💾 ratio + full eigenvalues 둘 다 저장
        results.setdefault(x_val, []).append({
            'ratio': ratio,
            'eigenvalues': val
        })

with open("results_curvature_concentration.pkl", "wb") as f:
    pickle.dump(results, f)

In [111]:
def _safe_extract_eigvals(obj):
    """
    Try to find an eigenvalue array (np.ndarray or list-like) inside obj.
    Return a 1D numpy array if found, else return None.
    Handles: np.ndarray, list/tuple of arrays, dict with keys like 'eigenvalues','eigvals','H','values',
    and nested structures.
    """
    # direct numpy array
    if isinstance(obj, np.ndarray):
        return obj.flatten()

    # numpy scalar or Python scalar -> not an eigenvalue array
    if isinstance(obj, (np.floating, np.integer, float, int, np.float64, np.int64)):
        return None

    # list/tuple -> try each element
    if isinstance(obj, (list, tuple)):
        for item in obj:
            arr = _safe_extract_eigvals(item)
            if arr is not None:
                return arr
        return None

    # dict -> try common keys first, then recurse over values
    if isinstance(obj, dict):
        # common possible keys that hold eigenvalues
        preferred_keys = ['eigenvalues', 'eigvals', 'eigvals_', 'H', 'values', 'vals', 'eigs', 'eigen']
        for k in preferred_keys:
            if k in obj:
                arr = _safe_extract_eigvals(obj[k])
                if arr is not None:
                    return arr

        # if none of preferred keys, recurse values (but skip scalar values)
        for v in obj.values():
            arr = _safe_extract_eigvals(v)
            if arr is not None:
                return arr
        return None

    # other types (pandas, torch tensor) - handle torch tensors
    try:
        import torch
        if isinstance(obj, torch.Tensor):
            return obj.detach().cpu().numpy().flatten()
    except Exception:
        pass

    # fallback: not recognized
    return None


def plot_cumulative_explained_ratio(results, alpha=0.9, save_path=None):
    """
    results: dict { log(n_params): [ either floats OR dicts with 'ratio' key ] }
    Draws median +/- std errorbar and saves figure.
    """
    xs = []
    medians = []
    stds = []
    for x_val, items in results.items():
        ratios = []
        # items may be list of floats or list of dicts or mixed
        for it in items:
            if isinstance(it, dict) and 'ratio' in it:
                try:
                    ratios.append(float(it['ratio']))
                except Exception:
                    pass
            elif isinstance(it, (float, int, np.floating, np.integer)):
                ratios.append(float(it))
            else:
                # maybe nested dict with ratio inside
                if isinstance(it, dict):
                    for v in it.values():
                        if isinstance(v, (float, int, np.floating, np.integer)):
                            ratios.append(float(v))
                            break
        if len(ratios) == 0:
            # skip if nothing found
            print(f"⚠️  No ratio entries found for x={x_val}, skipping.")
            continue
        xs.append(float(x_val))
        medians.append(np.median(ratios))
        stds.append(np.std(ratios))

    if len(xs) == 0:
        print("⚠️ No ratio data found in results. Nothing to plot.")
        return

    xs = np.array(xs)
    order = np.argsort(xs)
    xs = xs[order]
    medians = np.array(medians)[order]
    stds = np.array(stds)[order]

    plt.figure(figsize=(8,6))
    plt.errorbar(xs, medians, yerr=stds, fmt='o-', capsize=5, ecolor='gray', elinewidth=1.5)
    plt.xlabel("log(Number of parameters)")
    plt.ylabel(f"Cumulative explained ratio (α={alpha})")
    plt.title("Model Complexity vs. Curvature Concentration")
    plt.grid(True, alpha=0.3)

    os.makedirs("figs", exist_ok=True)
    if save_path is None:
        save_path = f"figs/cumulative_explained_ratio_{time.strftime('%Y%m%d_%H%M%S')}.png"
    else:
        # ensure directory exists
        d = os.path.dirname(save_path)
        if d:
            os.makedirs(d, exist_ok=True)

    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✅ Saved cumulative ratio plot → {save_path}")

def plot_logdet_vs_tracebound(results, save_path=None, clip_eps=1e-8):
    """
    results: dict { log(n_params): [ items ] }
    Each item can be:
      - a dict containing 'eigenvalues' (or 'eigvals', 'H', etc.)
      - a direct np.ndarray of eigenvalues
      - nested lists/dicts
    This computes per-trial:
      logdet = sum(log(clipped_eigvals))
      trace_bound = d * log(mean(clipped_eigvals))
    Then plots median +/- std across trials for each x (log n_params).
    """
    xs = []
    logdet_medians = []
    logdet_stds = []
    trace_medians = []
    trace_stds = []

    for x_val, items in results.items():
        # collect all eig arrays found under this x_val
        eig_lists = []
        for it in items:
            arr = _safe_extract_eigvals(it)
            if arr is None:
                continue
            # convert to numpy and flatten
            try:
                arr = np.array(arr, dtype=float).flatten()
            except Exception:
                continue
            eig_lists.append(arr)

        if len(eig_lists) == 0:
            print(f"⚠️ No eigenvalue arrays found for x={x_val}, skipping.")
            continue

        logdets = []
        tracebounds = []
        for eig in eig_lists:
            # sanitize values
            eig = eig[np.isfinite(eig)]
            if eig.size == 0:
                continue
            eig = np.clip(eig, clip_eps, None)  # avoid non-positive
            d = eig.size
            if d == 0:
                continue
            logdet = np.sum(np.log(eig))
            tracebound = d * np.log(np.sum(eig) / d)
            logdets.append(logdet)
            tracebounds.append(tracebound)

        if len(logdets) == 0:
            print(f"⚠️ After sanitization no valid eigvals for x={x_val}, skipping.")
            continue

        xs.append(float(x_val))
        logdet_medians.append(np.median(logdets))
        logdet_stds.append(np.std(logdets))
        trace_medians.append(np.median(tracebounds))
        trace_stds.append(np.std(tracebounds))

    if len(xs) == 0:
        print("⚠️ No valid data to plot for logdet vs tracebound.")
        return

    xs = np.array(xs)
    order = np.argsort(xs)
    xs = xs[order]
    logdet_medians = np.array(logdet_medians)[order]
    logdet_stds = np.array(logdet_stds)[order]
    trace_medians = np.array(trace_medians)[order]
    trace_stds = np.array(trace_stds)[order]

    plt.figure(figsize=(8,6))
    plt.errorbar(xs, logdet_medians, yerr=logdet_stds, fmt='o-', label='log|H| (∑ log λ_i)')
    plt.errorbar(xs, trace_medians, yerr=trace_stds, fmt='s--', label='trace bound (d·log(tr/d))')
    plt.xlabel("log(Number of parameters)")
    plt.ylabel("Value")
    plt.title("Log-determinant vs. Trace Bound (Curvature Scale)")
    plt.legend()
    plt.grid(True, alpha=0.3)

    os.makedirs("figs", exist_ok=True)
    if save_path is None:
        save_path = f"figs/logdet_vs_tracebound_{time.strftime('%Y%m%d_%H%M%S')}.png"
    else:
        d = os.path.dirname(save_path)
        if d:
            os.makedirs(d, exist_ok=True)

    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✅ Saved logdet/trace plot → {save_path}")

with open("results_curvature_concentration.pkl", "rb") as f:
    results = pickle.load(f)
plot_logdet_vs_tracebound(results, save_path='logdet_vs_tracebound.png')
plot_cumulative_explained_ratio(results, alpha=0.9, save_path='cumulative_explained_ratio.png')

✅ Saved logdet/trace plot → logdet_vs_tracebound.png
✅ Saved cumulative ratio plot → cumulative_explained_ratio.png


Task 2